In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

test_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en",split='test',streaming=True)

In [ ]:
train_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en",split='train',streaming=True)

In [ ]:
test_dataset

In [ ]:
train_dataset

In [ ]:
!pip3 install transformers==4.3.0 sentencepiece==0.1.95 nltk==3.5 protobuf==3.15.3 torch

In [6]:
from transformers import MarianTokenizer, MarianMTModel
mname = "marefa-nlp/marefa-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
#input = "President Putin went to the presidential palace in the capital, Kiev"

#translated_tokens = model.generate(**tokenizer.prepare_seq2seq_batch([input], return_tensors="pt"))
#translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
#print(translated_text)

In [7]:
new_train_dataset=[]
count=0
for item in train_dataset:
   count+=1
   if count<1000:
     translated_tokens = model.generate(**tokenizer.prepare_seq2seq_batch([item['sentence']], return_tensors="pt"))
     translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
     new_train_dataset.append({'audio':item['audio'],'sentence':translated_text})
   else:
      break


In [ ]:
new_train_dataset

In [9]:
from torch.utils.data import IterableDataset

class ListDataset(IterableDataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __iter__(self):
        return iter(self.data_list)
    def cast_column(self, column_name, data_type):
        for item in self.data_list:
            item[column_name] = data_type(item[column_name], sampling_rate=16000)
        return self
    def map(self, function):
        self.data_list = list(map(function, self.data_list))
        return self

new_train_dataset = ListDataset(new_train_dataset)


In [10]:
new_train_dataset

In [11]:
new_test_dataset=[]
count=0
for item in test_dataset:
   count+=1
   if count<250:
     translated_tokens = model.generate(**tokenizer.prepare_seq2seq_batch([item['sentence']], return_tensors="pt"))
     translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
     new_test_dataset.append({'audio':item['audio'],'sentence':translated_text})
   else:
      break


In [ ]:
new_test_dataset

In [13]:
from torch.utils.data import IterableDataset
from datasets import Value
class ListDataset(IterableDataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __iter__(self):
        return iter(self.data_list)
    def cast_column(self, column_name, data_type):
        for item in self.data_list:
            item[column_name] = data_type(item[column_name], sampling_rate=16000)
        return self

    def map(self, function):
        self.data_list = list(map(function, self.data_list))
        return self

new_test_dataset = ListDataset(new_test_dataset)

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Arabic", task="translate")

In [16]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Arabic", task="translate")

In [17]:
from datasets import Audio
class AudioWrapper:
    def __init__(self, audio_data, sampling_rate):
        self.audio_data = audio_data
        self.sampling_rate = sampling_rate

new_train_dataset = new_train_dataset.cast_column("audio", AudioWrapper)

In [18]:
import numpy as np
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]


    raw_speech_data = audio.audio_data

    # Check if raw_speech_data is a dictionary and access the specific key if needed
    if isinstance(raw_speech_data, dict):
        raw_speech_data = raw_speech_data.get("array", None)

    # Check if raw_speech_data is a valid numeric type
    if isinstance(raw_speech_data, (int, float, np.number, np.ndarray)):
        # compute log-Mel input features from input audio array
        input_features = feature_extractor(raw_speech_data, sampling_rate=audio.sampling_rate).input_features[0]


        encoded_text = tokenizer(
            batch["sentence"],
            padding=True,
            truncation=True,
            return_tensors="np",
            max_length=10
        )


        labels = encoded_text.input_ids.flatten()

        batch["input_features"] = input_features
        batch["labels"] = labels
    else:

        print("Invalid raw speech data:", raw_speech_data)

    return batch

In [19]:
new_train_dataset = new_train_dataset.map(prepare_dataset)

In [ ]:
#new_test_dataset = new_test_dataset.map(prepare_dataset)

In [20]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
!pip install evaluate

In [ ]:
!pip install jiwer

In [25]:
import evaluate

metric = evaluate.load("wer")

In [26]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
!pip install accelerate -U

In [32]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ar",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [29]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [33]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=new_train_dataset,
    eval_dataset=new_test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [34]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="arabic", task="translate")
ds = load_dataset("common_voice", "en", split="test", streaming=True)
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
input_speech = next(iter(ds))["audio"]
input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
